# **American Express - Default Prediction**

### **Setup**

In [1]:
import vaex
vaex.multithreading.thread_count_default = 8
import vaex.ml

import pandas as pd
import numpy as np
import os, gc, psutil, glob

### **Utility Functions**

In [2]:
def remove_output_files(file_pattern):
    fileList = glob.glob(file_patter)
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file: ", filePath)

In [3]:
def fill_and_convert_floats(ddf):
    for c in ddf.columns:
        if ddf[c].dtype == 'float64':
            ddf[c] = ddf[c].fillna(0.0).astype('float32')
    return ddf

def encode_cat_features(df):
    cat_features = ['D_63','D_64']
    label_encoder = vaex.ml.LabelEncoder(features=cat_features)
    df = label_encoder.fit_transform(df)
    df.drop(cat_features, inplace=True)
    df.rename('label_encoded_D_63','D_63')
    df.rename('label_encoded_D_64','D_64')
    df['D_64'] = df['D_64'].astype('float32')
    df['D_63'] = df['D_63'].astype('float32')
    df['B_31'] = df['B_31'].astype('float32')
    return df

def get_last_statement(df):
    return df.groupby(['customer_ID']).agg({col: vaex.agg.last(col) for col in df.get_column_names() if col not in ["customer_ID"]})

In [4]:
def get_last_statement_ex(df_test):
    delinquency_features = [col for col in df_test if col.startswith('D_')] 
    df = df_test.groupby(['customer_ID']).agg({col: vaex.agg.last(col) for col in df_test.get_column_names() if col not in delinquency_features + ["customer_ID"]})
    df.export_hdf5('./last-statement-p1.hdf5')
    del df
    gc.collect()
    delinquency_features = ['S_2'] + [col for col in df_test if col.startswith('D_') and len(col) == 4] 
    delinquency_features2 = ['S_2'] + [col for col in df_test if col.startswith('D_') and len(col) == 5]
    df_2 = df_test.groupby(['customer_ID']).agg({col: vaex.agg.last(col) for col in df_test.get_column_names() if col in delinquency_features})
    df_2.export_hdf5('./last-statement-p2.hdf5')
    del df_2
    gc.collect()
    df_3 = df_test.groupby(['customer_ID']).agg({col: vaex.agg.last(col) for col in df_test.get_column_names() if col in delinquency_features2})
    df_3.export_hdf5('./last-statement-p3.hdf5')
    del df_3
    gc.collect()
    last_statement_p1 = vaex.open('./last-statement-p1.hdf5')
    last_statement_p2 = vaex.open('./last-statement-p2.hdf5')
    last_statement_p3 = vaex.open('./last-statement-p3.hdf5')
    last_statement_p1 = last_statement_p1.drop('S_2')
    last_statement_p2 = last_statement_p2.drop('S_2')
    last_statement_p3 = last_statement_p3.drop('S_2')
    gc.collect()
    last_statement_p1 = last_statement_p1.join(last_statement_p2, how="inner", on='customer_ID')
    df_last_statement = last_statement_p1.join(last_statement_p3, how="inner", on='customer_ID')
    del last_statement_p1
    del last_statement_p2
    del last_statement_p3
    gc.collect()
    statement_path = './last-statement-p*.hdf5'
    remove_output_files(statement_path)
    return df_last_statement


In [5]:
def process_data(data):
    for i, df in enumerate(vaex.from_csv(f'../input/amex-default-prediction/{data}.csv', chunk_size=500_000)):
        df['S_2'] = df['S_2'].str.replace('-','').astype('float32')
        #df['R_26'] = df['R_26'].astype('int16')
        df = fill_and_convert_floats(df)
        df = encode_cat_features(df)
        df = get_last_statement(df)
        export_path = f'./{data}_{i:02}.hdf5'    
        df.export_hdf5(export_path)
        del df
        gc.collect()
    import_path = f'./{data}_*.hdf5'
    df = vaex.open(import_path)
    df.export_hdf5(f'./{data}.hdf5')
    del df
    gc.collect()
    remove_output_files(import_path)

In [6]:
def process_data_level2(df, data, flag):
    if flag == 1:
        df = get_last_statement_ex(df)     
    else:
        df = get_last_statement(df)
        df.drop('S_2', inplace=True)   
    label_encoder = vaex.ml.LabelEncoder(features=['customer_ID'])
    df = label_encoder.fit_transform(df)
    df_customer_map = df[['label_encoded_customer_ID', 'customer_ID']]
    df.drop('customer_ID', inplace=True)
    df.rename('label_encoded_customer_ID','customer_ID')
    df_customer_map.export_hdf5(f'./{data}_customer_map.hdf5')
    df.export_hdf5(f'./{data}v2.hdf5')
    del df
    del df_customer_map
    gc.collect()

In [7]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [8]:
def amex_metric_np(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric_np(y_true, y_pred), True

### **Preprocessing - Level 1 - Reduce Train & Test Datasets**

Train & Test datasets will be processed to reduce the size so we can perform feature engineering and build models without memory, CPU and hard disk constraints

In [9]:
df_train = vaex.open('../input/amex-prediction-starter-level-2/train_datav2.hdf5')
df_test = vaex.open('../input/amex-prediction-starter-level-2/test_datav2.hdf5')
df_train_map = vaex.open('../input/amex-prediction-starter-level-2/train_data_customer_map.hdf5')
df_test_map = vaex.open('../input/amex-prediction-starter-level-2/test_data_customer_map.hdf5')

### **Prepare Data to Train Model**

In [10]:
df_train_labels = vaex.open('../input/amex-default-prediction/train_labels.csv')
df_train_labels = df_train_labels.join(df_train_map, how="inner", on="customer_ID")

all_features = [col for col in df_train]

df_customer = df_train[all_features]
df_customer = df_customer.join(df_train_labels, left_on='customer_ID', right_on='label_encoded_customer_ID', how='inner')
df_customer.drop(['label_encoded_customer_ID'], inplace=True)

#,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,D_63,D_64,customer_ID,target
0,0.7442702,0.0030916873,0.0072427043,1.007496,0.009336631,0.29910848,0.003750894,0.0022871876,0.0,0.0,0.0037418173,0.05836785,0.8853801,0.012688401,0.0045256414,0.3764425,0.63416755,0.07867742,0.0,0.12910235,0.04106728,0.003342006,0.16706242,0.33532014,0.0043627215,0.006509866,0.23279026,0.5838689,0.29861808,0.008751293,0.0030340385,0.0017301164,0.0073325606,1.0094647,0.0068011335,0.25435725,0.010460004,0.0054463046,0.12951766,0.5913745,0.002919833,0.004855534,0.009396881,0.0,0.009415094,0.0694069,0.000982847,0.07935046,0.008964529,0.28673884,0.44811037,0.0024738235,0.0053694546,0.0,1.009645,0.00922647,0.0,0.0021670526,6.0,0.19010633,0.0057525164,0.005201095,0.006982812,0.001905763,0.006499635,0.007607226,0.0074345022,0.0051399865,0.5058207,0.025566528,0.0,0.002268651,0.00041639243,0.0056640273,0.0,0.001640105,0.009689874,0.42343187,0.0028987455,0.0035675801,0.0056366734,0.0007157872,0.009354974,0.0,0.005058291,0.00038713944,0.009863765,0.008546336,0.009050531,0.0009270649,0.5031435,0.006148745,1.0063536,0.07451593,0.0036630714,0.005190503,0.000863827,0.0016088556,0.008536328,0.005483388,0.0,0.0,0.006537585,0.00901568,0.0,0.0071298457,0.0044488194,0.0,1.0,0.0049272077,0.0021550814,0.0028910656,0.005200902,0.006891079,0.000495353,1.0023797,0.0020826089,0.00035686433,0.0030126462,0.003240912,0.008255401,0.006943732,0.0004970511,0.0062739803,0.0049009165,0.0014250554,0.041788224,0.13216524,0.0052063847,0.9776156,0.004089647,0.0042449883,0.0008384559,0.0011774149,0.0,0.0,0.0008451528,0.0018779807,0.004563586,0.0,1.0055594,1.0,0.0,0.009942509,0.0,0.0,0.0,1.0051179,0.031578425,0.0,0.0009948038,0.0,0.81625384,0.0,6.0,0.80700576,0.81417465,0.0,0.7262038,0.57253176,0.0040466255,0.22812885,0.0029846623,1.0,1.0007654,1.0038574,1.0015905,0.009653939,0.0,0.0011981075,0.0061936122,0.0,0.009960352,0.009150347,0.0,0.0,0.0,0.0,0.0,0.004333312,0.009860008,0.0029108082,0.0,0.0022399465,0.0022530325,0.004805641,3.0,3.0,0,0
1,0.8260247,0.23716606,0.0202482,1.0092281,0.0083649205,0.09630558,0.006065481,0.017009411,0.0,0.008889698,0.0036192448,0.048056666,0.28356326,0.019710377,0.005484134,0.5763002,0.5246397,0.0965258,0.0,0.093564026,0.1049171,0.00026890857,0.06650882,0.33795607,0.008917741,0.005071848,0.13925605,0.5259909,0.0565117,0.0,0.012162755,0.0028940304,0.0057552657,1.0015627,0.0060377787,0.06585398,0.023065902,0.5995024,0.09448304,0.19597995,0.027668525,0.0036475766,0.32277966,0.01047724,0.008190892,0.07101729,0.034526795,0.3327187,0.0073206285,0.36197263,0.24421707,0.008175531,0.33755976,0.008135456,0.6533203,0.0025766133,1.0,0.003182337,6.0,0.6563971,0.0016308156,0.007856963,0.0049321796,0.0016360391,0.006218958,0.009333168,0.01503285,0.0044198954,0.10441527,0.09113425,0.0,0.007612308,0.14299467,0.14174816,0.0,0.009908457,0.0022537925,0.22737052,0.013268213,0.009658001,0.008888395,0.004303078,0.0015333936,0.0,0.009798362,0.20292905,0.0010397487,0.0028578353,0.003293858,0.002790434,0.0,0.0018962498,1.0042117,0.06958941,0.008083812,0.0011859015,0.009549554,0.00296493,0.0021200862

### **Convert Vaex DataFrames to Pandas DataFrames**

In [11]:
df_train = df_train.to_pandas_df()
df_test = df_test.to_pandas_df()
df_train_map = df_train_map.to_pandas_df()
df_test_map = df_test_map.to_pandas_df()

df_train_labels = df_train_labels.to_pandas_df()
df_customer = df_customer.to_pandas_df()

### **Train Model**

In [12]:
import lightgbm as lgb
from lightgbm import log_evaluation
from sklearn.model_selection import train_test_split

In [13]:
y = df_customer.pop('target')
model_features = [col for col in df_customer]
X = df_customer[model_features]

In [14]:
X.shape

(458913, 189)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1, random_state=42)

In [16]:
dtrain = lgb.Dataset(
    data=X_train,
    label=y_train
)

dvalid = lgb.Dataset(
    data=X_test,
    label=y_test,
    reference=dtrain
)

In [17]:
categorical_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
df_customer['D_117'] = df_customer['D_117'] + 1
df_customer['D_126'] = df_customer['D_126'] + 1

In [18]:
lgb_params={
    "objective": "binary",
    "n_estimators": 1200,
    "learning_rate": 0.03,
    "reg_lambda": 50,
    "min_child_samples": 2400,
    'max_depth': 8,
    "num_leaves": 255,
    "colsample_bytree": 0.19,
    'verbose': -1,
    "random_state": 1
}
    
model = lgb.train(
    params=lgb_params,
    train_set=dtrain,
    valid_sets=[dvalid],
    feval=lgb_amex_metric,
    callbacks=[log_evaluation(100)]
)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


[100]	valid_0's binary_logloss: 0.0385432	valid_0's amex_metric: nan
[200]	valid_0's binary_logloss: 0.013433	valid_0's amex_metric: nan
[300]	valid_0's binary_logloss: 0.0104473	valid_0's amex_metric: nan
[400]	valid_0's binary_logloss: 0.0106828	valid_0's amex_metric: nan
[500]	valid_0's binary_logloss: 0.0108432	valid_0's amex_metric: nan
[600]	valid_0's binary_logloss: 0.0116766	valid_0's amex_metric: nan
[700]	valid_0's binary_logloss: 0.0114033	valid_0's amex_metric: nan
[800]	valid_0's binary_logloss: 0.0115505	valid_0's amex_metric: nan
[900]	valid_0's binary_logloss: 0.0116415	valid_0's amex_metric: nan
[1000]	valid_0's binary_logloss: 0.0113862	valid_0's amex_metric: nan
[1100]	valid_0's binary_logloss: 0.0115424	valid_0's amex_metric: nan
[1200]	valid_0's binary_logloss: 0.0129676	valid_0's amex_metric: nan


In [19]:
gc.collect()

519

In [20]:
y_pred = model.predict(X_test)

In [21]:
y_pred

array([0.01288388])

### **Make Predictions**

In [22]:
model_features = [col for col in df_customer]
df_customer_test = df_test[model_features]

In [23]:
df_customer_test_pred = model.predict(df_customer_test)

In [24]:
df_customer_test_pred

array([0.0141366 , 0.00077073, 0.00850981, ..., 0.34175355, 0.70307158,
       0.01402809])

In [25]:
df_customer_test

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_139,D_140,D_141,D_142,D_143,D_144,D_145,D_63,D_64,customer_ID
0,0.794581,0.000953,0.005632,0.810286,0.004518,0.249819,0.008213,0.004759,0.00000,0.093510,...,0.005481,0.000222,0.009534,0.000000,0.006538,0.001135,0.000323,0.0,2.0,0
1,0.943439,0.003285,0.013842,1.000161,0.002857,0.128938,0.008345,0.008910,0.00412,0.000000,...,1.007859,0.009740,0.924881,0.522326,1.006161,0.648153,0.642873,3.0,3.0,1
2,0.764847,0.001760,0.003702,0.810497,0.008001,0.000000,0.009685,0.001857,0.00000,0.103272,...,0.008729,0.002711,0.005979,0.000000,0.007977,0.003958,0.000977,0.0,1.0,2
3,0.610121,0.771469,0.613606,0.022343,0.003864,0.174504,0.008826,0.733658,0.00000,0.449111,...,1.006573,0.005742,0.907650,0.201316,1.004309,0.139868,0.098931,0.0,2.0,3
4,0.882037,0.005513,0.006477,0.819673,0.002524,0.000000,0.007767,0.000086,0.00000,0.000000,...,0.001783,0.008707,0.008966,0.000000,0.008267,0.005799,0.007622,0.0,2.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,0.679791,0.000315,0.025005,0.496837,0.251907,0.142065,0.003219,0.149840,0.00000,0.626851,...,1.005299,0.000072,1.036161,0.803950,1.001329,0.626038,0.096816,0.0,3.0,924616
924617,0.767409,0.004265,0.053714,0.321444,0.259923,0.336231,0.093263,0.258417,0.00000,0.000000,...,0.000975,0.009754,0.007228,0.000000,0.002002,0.003100,0.004315,3.0,2.0,924617
924618,0.417093,0.008597,0.002355,0.817596,0.006410,0.163120,0.008076,0.005141,0.00000,0.705349,...,1.006942,0.007578,0.865715,0.034273,1.006945,0.450350,0.099008,0.0,2.0,924618
924619,0.454026,0.155667,0.667262,0.030093,0.252782,0.171745,0.007521,0.808719,0.00000,0.039547,...,0.005669,0.006214,0.007119,0.000000,0.006731,0.005600,0.008319,0.0,1.0,924619


In [26]:
df_customer_test = pd.merge(df_customer_test, df_test_map, how="inner", left_on="customer_ID", right_on="label_encoded_customer_ID")

In [27]:
df_customer_test = df_customer_test.drop(columns={'customer_ID_x','label_encoded_customer_ID'})

In [28]:
df_customer_test_pred = pd.DataFrame(df_customer_test_pred.tolist())

In [29]:
df_customer_test['prediction'] = df_customer_test_pred

In [30]:
df_customer_test

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_140,D_141,D_142,D_143,D_144,D_145,D_63,D_64,customer_ID_y,prediction
0,0.794581,0.000953,0.005632,0.810286,0.004518,0.249819,0.008213,0.004759,0.00000,0.093510,...,0.000222,0.009534,0.000000,0.006538,0.001135,0.000323,0.0,2.0,43906b6773ec5fb8cf4452ceec799976493b47fc579d4b...,0.014137
1,0.943439,0.003285,0.013842,1.000161,0.002857,0.128938,0.008345,0.008910,0.00412,0.000000,...,0.009740,0.924881,0.522326,1.006161,0.648153,0.642873,3.0,3.0,4390cd51fe9c61555bb344c39ff42fcae135e2bfccadd9...,0.000771
2,0.764847,0.001760,0.003702,0.810497,0.008001,0.000000,0.009685,0.001857,0.00000,0.103272,...,0.002711,0.005979,0.000000,0.007977,0.003958,0.000977,0.0,1.0,43946d3f3e74a62b41a17a43fbccc1457da3bce191658c...,0.008510
3,0.610121,0.771469,0.613606,0.022343,0.003864,0.174504,0.008826,0.733658,0.00000,0.449111,...,0.005742,0.907650,0.201316,1.004309,0.139868,0.098931,0.0,2.0,43958550b73643ac8d309ff22fbcb4ce0a23a0a914f437...,0.411592
4,0.882037,0.005513,0.006477,0.819673,0.002524,0.000000,0.007767,0.000086,0.00000,0.000000,...,0.008707,0.008966,0.000000,0.008267,0.005799,0.007622,0.0,2.0,4396f71c2a23f3ea73802cd79133adf767101ebee636f8...,0.003120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,0.679791,0.000315,0.025005,0.496837,0.251907,0.142065,0.003219,0.149840,0.00000,0.626851,...,0.000072,1.036161,0.803950,1.001329,0.626038,0.096816,0.0,3.0,fff5ba5c7925c14b17b13cf2a8db0b9221df47d7caabbe...,0.048898
924617,0.767409,0.004265,0.053714,0.321444,0.259923,0.336231,0.093263,0.258417,0.00000,0.000000,...,0.009754,0.007228,0.000000,0.002002,0.003100,0.004315,3.0,2.0,fff60f216e5b5df086502d86a04392050e67ee77d5e162...,0.109117
924618,0.417093,0.008597,0.002355,0.817596,0.006410,0.163120,0.008076,0.005141,0.00000,0.705349,...,0.007578,0.865715,0.034273,1.006945,0.450350,0.099008,0.0,2.0,fff9bb02cbd43adf24d7a5a13ce6c6ce9910c9991f9b4c...,0.341754
924619,0.454026,0.155667,0.667262,0.030093,0.252782,0.171745,0.007521,0.808719,0.00000,0.039547,...,0.006214,0.007119,0.000000,0.006731,0.005600,0.008319,0.0,1.0,fffd9644010686e7e2bf96ca6f3af174bd2b075a887445...,0.703072


In [31]:
df_customer_test = df_customer_test.rename(columns={"customer_ID_y": "customer_ID"})

In [32]:
final_prediction = df_customer_test[["customer_ID", "prediction"]]

### **Save File**

In [33]:
final_prediction.to_csv("./submission.csv",index=False)